-------------------------------------------------------------------

- **Aluno:** Renato Andrade Mosqueira Furtado
- **Matrícula:** 102100652
- **e-mail:** renato.andrade@engenharia.ufjf.br
- **Disciplina:** Planejamento de Sistemas Elétricos
- **Objetivo:** formular genericamente um problema de otimização a fim de estimar as vazões para qualquer usina hidrelétrica.

-------------------------------------------------------------------

# **Instalação do PySDDP**

In [ ]:
!pip install PySDDP

# **Importação do dados e bibliotecas**

- Leitura do deck de dados disponibilizados no site do CCEE na pasta alocada no drive do usuário. Deve-se, portanto, trocar a variável **CAMINHO** colocando o endereço de acesso à pasta.

In [ ]:
from google.colab import drive
import os
from google.colab import files


drive.mount('/content/gdrive')

print(os.getcwd())


#============== Usuário deve trocar aqui de acordo com a pasta utilizada no drive ================
Caminho = 'gdrive/My Drive/Colab Notebooks/Planejamento Energético/Dados Newave/'
#=================================================================================================

Mounted at /content/gdrive
/content


In [ ]:
from PySDDP.Pen import Newave
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from cvxopt import matrix,solvers

In [ ]:
casoestudo = Newave(Caminho)

OK! Leitura do CASO.DAT realizada com sucesso.
OK! Leitura do ARQUIVOS.DAT realizada com sucesso.
OK! Leitura do DGER.DAT realizada com sucesso.
OK! Leitura do HIDR.DAT realizada com sucesso.
OK! Leitura do VAZOES.DAT realizada com sucesso.
OK! Leitura do MODIF.DAT realizada com sucesso. ( 99 Usinas Hidraulicas Modificadas )
OK! Leitura do EXPH.DAT realizada com sucesso. ( 2 Usinas Hidraulicas Expandidas )
OK! Leitura do CONFHD.DAT realizada com sucesso.
OK! Leitura do REE.DAT realizada com sucesso. ( 12 Reservatórios Equivalentes de Energia Foram Lidos )
OK! Leitura do SISTEMA.DAT realizada com sucesso.


# **Leitura da usina e dados da otimização**

- **Dados de entrada preenchidos pelo usuário:**
 - *Nome_usina*:deve-se colocar o nome da usina a qual o usuário deseja estudar;
 - *Ordem*: corresponde à quantidade de coeficientes que se deseja aplicar a fim de estimar as vazões (**ESSE NÚMERO DEVE VARIAR DE 1 até 12**);

 - *Mes_estimado*: variável do tipo *string* que indica o mês cujos coeficientes serão determinados para estimar as vazões;
 - *Ano_inicial*: variável que indica a partir de qual ano será feita o cálculo dos coeficientes. Em outras palavras, indica o ano a partir do qual as vazões em todos os meses são conhecidas;
 - *Ano_final*: variável que indica qual o último ano a ser utilizado para cálculo dos coeficientes, ou seja, o último ano no qual todas as vazões mensais são conhecidas.


- **OBS:**

  - a variável *Ordem* deve ser um valor entre 1 e 11;
  - a variável *Ano_inicial* deve ser maior ou igual a 1932;
  - a variável *Ano_final* deve ser menor ou igual ao ano presente desde que o mês que se deseja estimar os coeficientes já possua uma vazão determinada;
  - **Para evitar problemas de dimensionamento de matrizes, é recomendado que o número de variáveis (número de anos + ordem) esteja condizente com o número de equações de igualdade (número de anos).** Exemplo: analisar de 2021 até 2022 com 7 coeficientes (ordem 7), dá erro de condicionamento matricial.

## Função para cálculo dos parâmetros

In [ ]:
def parametros_vazao(Nome_usina,Ordem,Mes_estimado,Ano_inicial,Ano_final):

  #------------------ Dados de entrada ----------------------

  Nome_usina = Nome_usina
  Ordem = Ordem
  Mes_estimado = Mes_estimado
  Ano_inicial = Ano_inicial
  Ano_final = Ano_final

  Nr_anos = Ano_final - Ano_inicial + 1

  #---------------- Leitura da usina selecionada ------------

  uhe = casoestudo.confhd.get(Nome_usina) # Nome_usina -> entrada pelo usuário

  vazoes = np.array(uhe['vazoes'],dtype=float) # Acessa as vazões da usina em questão

  data_vazoes = pd.DataFrame(data=vazoes,columns=['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'],index = np.linspace(int(1931),int(2023),93))
  # Transforma em DataFrame no qual as colunas são os meses e as linhas os anos: 1931 - 2023

  #--------------- Criação de uma lista com meses -----------------------------

  mes_lista = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']

  #--------------- Desvio padrão e média de todas as vazões do mês selecionado ---------

  desvio = np.std(data_vazoes.loc[Ano_inicial:Ano_final,Mes_estimado])
  media = np.mean(data_vazoes.loc[Ano_inicial:Ano_final,Mes_estimado])

  #==================================================================================================
  #========================== MONTAGEM DO PROBLEMA DE OTIMIZAÇÃO ====================================
  #==================================================================================================


  # Número de variáveis = Nr_anos + Ordem

  Nvar = Nr_anos + Ordem

  # 1) Matriz P

    # - Erro de cada ano recebe valor 2 na diagonal seguindo a lógica do cvxopt quadratic

  P = np.zeros((Nvar,Nvar))

  for k in range(0,Nr_anos):
    P[k][k] = 2.

  P = matrix(P) # x [erro1 erro2 ... erro_nranos Fi1 Fi2 Fi3... Fi_ordem]

  # 2) Matriz q

    # - Deve ser uma matriz de zeros pois a FOB não tem uma parte linear

  q = matrix(np.zeros((Nvar,1)))

  # 3) Matriz A

  A_diag = np.identity(Nr_anos) # Matriz identidade referente aos erros de cada ano

  A_vaz = np.zeros((Nr_anos,Ordem)) # Mariz com as vazões mensais de cada ano de acordo com a ordem

  Nr_mes_estimado = 0

  for k in range(0,len(mes_lista)):
    if mes_lista[k]==Mes_estimado:
      Nr_mes_estimado = k+1

  for linha in range(0,Nr_anos):

    ano = Ano_inicial + linha # acessar o DataFrame

    coluna = 0
    for ord in range(Nr_mes_estimado - Ordem - 1,Nr_mes_estimado-1):

      if ord<0:

        ano_anterior = ano - 1

        mes_ano_anterior = 13 + ord

        A_vaz[linha][coluna] = float(data_vazoes[mes_lista[mes_ano_anterior-1]][ano_anterior])

        coluna = coluna + 1

      else:

        A_vaz[linha][coluna] = float(data_vazoes[mes_lista[ord]][ano])

        coluna = coluna + 1


  A = matrix(np.concatenate((A_diag,A_vaz),axis = 1))


  # 4) Vetor coluna b (restrição de igualdade) - vazões do mês que se deseja estimar

  b = np.zeros((Nr_anos,1))

  for k in range(0,Nr_anos):

    ano = Ano_inicial + k

    b[k][0] = float(data_vazoes[Mes_estimado][ano])

  b = matrix(b)

  # 5) G e h -> não há restrições de desigualdade

  G_menor_igual = np.zeros((Nr_anos,Nvar))
  G_maior_igual = np.zeros((Nr_anos,Nvar))

  for k in range(0,Nr_anos):
    G_menor_igual[k][k] = 1
    G_maior_igual[k][k] = -1

  G = matrix(np.concatenate((G_menor_igual,G_maior_igual),axis = 0))

  h = matrix((media + desvio)*np.ones((2*Nr_anos,1)))

  # 6) Solução do problema de otimização

  solucao = solvers.qp(P,q,G,h,A,b)

  #====================================== Resultados ==========================================

  # Coeficientes

  coef = [solucao['x'][i] for i in range(Nr_anos,Nvar)]

  meses_coef = list()

  for ord in range(Nr_mes_estimado - Ordem - 1,Nr_mes_estimado-1):

      if ord<0:

        mes_ano_anterior = 13 + ord

        meses_coef.append(mes_lista[mes_ano_anterior-1])

      else:

        meses_coef.append(mes_lista[ord])

  coeficientes = pd.DataFrame(data = [coef],columns=meses_coef)

  # Comparação entre o valor real, estimado e o erro (FORMA DE CONFERIR)

  valor_real = list(data_vazoes.loc[Ano_inicial:Ano_final,Mes_estimado])

  valor_estimado = np.dot(np.array(A_vaz),np.transpose(np.array(coef)))

  erro = [solucao['x'][i] for i in range(0,Nr_anos)]

  compara = pd.DataFrame(data = np.transpose([valor_real,valor_estimado,erro]),columns=['Valor real (m3/s)','Estimado (m3/s)','Erro (m3/s)'],index=np.linspace(Ano_inicial,Ano_final,Nr_anos))

  return coeficientes,compara,solucao['primal objective']


## Simulações

 - **Endereços para acessar os resultados:**

  - **0:** *DataFrame* com os coeficientes dos meses de acordo com a ordem;
  - **1:** *DataFrame* com os valores de vazões reais, estimadas e o erro advindo da otimização;
  - **2:** valor da função objetivo.

In [ ]:
resultado = parametros_vazao(Nome_usina = 'Furnas',Ordem = 6,Mes_estimado = 'Maio',Ano_inicial = 1932 ,Ano_final = 2022)

     pcost       dcost       gap    pres   dres
 0:  9.8499e+05 -5.8770e+07  6e+07  2e-16  5e-08
 1:  9.8499e+05  3.8744e+05  6e+05  2e-16  4e-09
 2:  9.8499e+05  9.7901e+05  6e+03  2e-16  2e-09
 3:  9.8499e+05  9.8493e+05  6e+01  2e-16  2e-09
 4:  9.8499e+05  9.8499e+05  6e-01  2e-16  1e-09
Optimal solution found.


In [ ]:
resultado[0]

,Novembro,Dezembro,Janeiro,Fevereiro,Março,Abril
0,-0.00954,0.07221,0.031649,0.088618,0.09511,0.299485


In [ ]:
resultado[1]

,Valor real (m3/s),Estimado (m3/s),Erro (m3/s)
1932.0,673.0,709.451882,-36.451882
1933.0,441.0,456.282884,-15.282884
1934.0,333.0,362.669483,-29.669483
1935.0,909.0,923.797355,-14.797355
1936.0,626.0,574.621337,51.378663
...,...,...,...
2018.0,245.0,303.901844,-58.901844
2019.0,524.0,502.780957,21.219043
2020.0,462.0,631.612288,-169.612288
2021.0,306.0,392.964031,-86.964031


In [ ]:
print(f'Valor da função objetivo = {resultado[2]} (m3/s)^2')

Valor da função objetivo = 984989.4474656186 (m3/s)^2
